## NI - MPI homework
author : hamplto3

### Solver class

In [4]:
import numpy as np

class mpiSolver():
    def __init__(self, gamma, jacobi=True):
        # stopping threshold
        self.thresh = 10**-6

        # matrix A
        self.A = np.zeros((20, 20), np.float64)

        # main diagonal
        np.fill_diagonal(self.A, gamma)

        # under main diagonal
        np.fill_diagonal(self.A[1:], -1)

        # above main diagonal
        np.fill_diagonal(self.A[:,1:], -1)

        # vector b
        self.b = np.zeros((20, 1), np.float64)

        # fill values
        self.b[1 : -1] = gamma - 2
        self.b[0] = self.b[-1]  = gamma -1

        # vector x
        self.x = np.zeros((20, 1), np.float64)

        # diagonal
        self.D = np.zeros((20, 20), np.float64)
        np.fill_diagonal(self.D, gamma)

        # under diagonal (lower)
        self.L = np.zeros((20, 20), np.float64)
        np.fill_diagonal(self.L[1:], -1)

        # above diagonal (upper)
        self.U = np.zeros((20, 20), np.float64)
        np.fill_diagonal(self.U[:,1:], -1)

        # regular matrix Q
        if jacobi:
            self.Q = self.D
        else:
            self.Q = self.D + self.L

        # inverse Q
        self.Q_inv = np.linalg.inv(self.Q)

    # check if method converges
    def __converges__(self):
        return all(np.linalg.eigvals(np.identity(20, np.float64) - np.dot(self.Q_inv, self.A)) < 1)

    # check if the threshold is satisfied
    # check if eukleid_norm(Ax - b) / eukleid_norm(b) < 10**-6
    def __satisfied__(self, x):
        return (np.linalg.norm(np.dot(self.A, x) - self.b) / np.linalg.norm(self.b)) < self.thresh

    # step in calculation
    def __step__(self, x):
        return np.dot(self.Q_inv, (np.dot(self.Q - self.A, x) + self.b))

    # main calculation function
    def __calculate__(self):
        iterations = 0

        while True:
            self.x = self.__step__(self.x)

            iterations += 1

            if self.__satisfied__(self.x):
                return self.x, iterations

    # main class function
    def run(self, show=True):
        if self.__converges__():
            res, iterations = self.__calculate__()

            if show:
                print(f'iterations: {iterations}')
                print("result x")
                print(res.T)

            return res, iterations
        else:
            print("method doesnt converge")
            return [], 0


    #################################################################
    # some experiments with convergence conditions

    # check if matrix is diagonally dominant
    def __diagonallyDominant__(self):
        # diagonal values
        diagonal = np.diag(np.abs(self.A))

        # sum of rest
        rest = []
        for idx, array in enumerate(np.abs(self.A)):
            rest.append(np.sum(np.delete(array, idx)))
        
        return all(diagonal > rest)

    """
    source for formulas : https://math.stackexchange.com/questions/270181/the-convergence-of-jacobi-and-gauss-seidel-methods
    author : https://math.stackexchange.com/users/49167/thomas
    date : 4.12.2021
    """
    # G = -D^-1 * (A - D)
    # spectral radius(G) < 1
    def __convergesJacobi__(self):
        return all(np.linalg.eigvals(np.dot(-1 * np.linalg.inv(self.D), (self.A - self.D))) < 1)

    # G = -(D + L)^-1 * U
    # spectral radius(G) < 1
    def __convergesGS__(self):
        return all(np.linalg.eigvals(np.dot(-1 * np.linalg.inv((self.D + self.L)), self.U)) < 1)

    # source : https://en.wikipedia.org/wiki/Jacobi_method
    # standard convergence condition (for any iterative method) is when the spectral radius of the iteration matrix is less than 1
    # spectral radius(Q_inv(L + U)) < 1
    def __convergesWiki__(self):
        return all(np.linalg.eigvals(np.dot(np.linalg.inv(self.D), (self.L + self.U))) < 1)

    # function to call all convergence conditions
    def run_all_convergence_tests(self):
        print('strict diagonally dominant')
        print(self.__diagonallyDominant__())
        print('spectral radius Jacobi')
        print(self.__convergesJacobi__())
        print('spectral radius GS')
        print(self.__convergesGS__())
        print('spectral radius from wiki')
        print(self.__convergesWiki__())
        print('MPI convergence')
        print(self.__converges__()) 


### How to run solver

In [5]:
# create a solver class, set gamma value and select method
# by default solver uses jacobi method, for GS use jacobi=False
mpi = mpiSolver(gamma=5, jacobi=True)

# run the iterative method
# by default show is set to True and it prints the result 
res, iterations = mpi.run(show=True)

# my experiments with different or semi-different convergence conditions
#mpi.run_all_convergence_tests()


iterations: 15
result x
[[0.99999979 0.99999958 0.99999941 0.99999925 0.99999915 0.99999905
  0.99999901 0.99999896 0.99999895 0.99999894 0.99999894 0.99999895
  0.99999896 0.99999901 0.99999905 0.99999915 0.99999925 0.99999941
  0.99999958 0.99999979]]


### Výsledky

| Metoda &nbsp; | Varianta &nbsp; | Iterace &nbsp; | Konverguje &nbsp;|
|:--------------:|:-----------:|:------------:|:--------:|
| Ja | $\gamma$ = 5 | 15 | konverguje |
| Jb | $\gamma$ = 2 | 987 | konverguje |
| Jc | $\gamma$ = $\frac{1}{2}$ | -1 | nekonverguje |
| GSa | $\gamma$ = 5 | 10 | konverguje |
| GSb | $\gamma$ = 2 | 495 | konverguje |
| GSc | $\gamma$ = $\frac{1}{2}$ | -1 | nekonverguje |

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4512f25c-ca03-431d-8d06-60b578a14173' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>